In [22]:
# Sets up the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configures the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
from pathlib import Path
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configures OS routines
import os
import re

# Configures the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from CRUD_Python_Module import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "sUchStr0ngP4S$w0rD"

# Connects to database via CRUD Module
db = AnimalShelter(username, password)

# Reads all documents
df = pd.DataFrame.from_records(db.read({}))

# Removes _id to prevent crash
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Adds species
if 'animal_type' in df.columns:
    df['species'] = df['animal_type'].apply(
        lambda x: 'Cat' if isinstance(x, str) and 'cat' in x.lower()
        else ('Dog' if isinstance(x, str) and 'dog' in x.lower() else None)
    )
else:
    df['species'] = None

# Rounds age weeks
if 'age_upon_outcome_in_weeks' in df.columns:
    df['age_upon_outcome_in_weeks'] = pd.to_numeric(
        df['age_upon_outcome_in_weeks'], errors='coerce'
    ).round().astype('Int64')

    
#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Logo setup
image_filename = 'Grazioso Salvare Logo.png'
if os.path.exists(image_filename):
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
    logo_img = html.A(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                 style={'height': '80px'}),
        href='https://www.snhu.edu',
        target='_blank'
    )
else:
    # Displays fallback text if image not found
    logo_img = html.A(
        html.Div("Grazioso Salvare (logo missing)", style={"fontWeight": "bold", "fontSize": "16px"}),
        href="https://www.snhu.edu",
        target="_blank",
        title="Go to SNHU Website"
    )
    print(f"Logo not found at: {logo_path}")

# Filter inputs
filter_radio = dcc.RadioItems(
    id='filter-type',
    options=[
        {'label': 'Water Rescue', 'value': 'water'},
        {'label': 'Mountain / Wilderness Rescue', 'value': 'mountain'},
        {'label': 'Disaster / Individual Tracking', 'value': 'disaster'},
        {'label': 'Reset (All Animals)', 'value': 'reset'},
    ],
    value='reset',
    labelStyle={'display': 'inline-block', 'margin-right': '20px'}
)

app.layout = html.Div([
    # Header
    html.Div([
        html.Div(logo_img, style={'flex': '0 0 auto'}),
        html.Div(html.B(html.H1('SNHU CS-340 Dashboard')), 
                 style={'flex': '1', 'textAlign': 'center', 'marginTop': '10px'}),
        html.Div(html.Div(style={'width': '80px'}), style={'flex': '0 0 auto'})
    ], style={'display': 'flex', 'alignItems': 'center', 
              'padding': '10px', 'borderBottom': '1px solid #ccc'}),

    # Identifier
    html.Div(html.P("Dashboard created by Arielle Moore – Hi, Cannoli!"),
             style={'fontStyle': 'italic', 'textAlign': 'center', 'marginTop': '6px'}),

    html.Hr(),

    # Filter Panel
    html.Div(
        children=[
            html.Div([
                html.Label("Interactive Filter Options:", 
                           style={'fontWeight': 'bold', 'marginBottom': '6px'}),
                filter_radio
            ], style={'padding': '12px'})
        ],
        style={'border': '2px solid #6a1b9a', 'borderRadius': '6px', 
               'backgroundColor': '#fff', 'marginLeft': '10px', 
               'marginRight': '10px'}
    ),
    html.Hr(),
    # Interactive Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[],
        style_table={'overflowX': 'auto', 'maxHeight': '400px'},
        style_cell={'textAlign': 'left', 'minWidth': '100px', 'whiteSpace': 'normal'},
    ),
    html.Br(),
    html.Hr(),
    # Graph and Map
    html.Div(className='row',
             style={'display': 'flex', 'marginLeft': '10px', 'marginRight': '10px'},
             children=[
                 html.Div(id='graph-id',
                          className='col s12 m6',
                          style={'flex': '1', 'padding': '10px'}),
                 html.Div(id='map-id',
                          className='col s12 m6',
                          style={'flex': '1', 'padding': '10px'})
             ])
])


#############################################
# Interaction Between Components / Controller
#############################################

# Helper to build rescue query
def build_rescue_query(rescue_type):

    if rescue_type == 'reset' or rescue_type is None:
        return {}

    # NOTE: We can externalise configuration instead of hardcoding if scaling
    rescue_map = {
        'water': {
            'breeds': [
                r'labrador\s*retriever', r'chesapeake\s*bay\s*retriever', r'newfoundland'
            ],
            'sex': r'intact\s*female',
            'min_age': 26,
            'max_age': 156
        },
        'mountain': {
            'breeds': [
                r'german\s*shepherd', r'alaskan\s*malamute', r'old\s*english\s*sheepdog',
                r'siberian\s*husky', r'rottweiler'
            ],
            'sex': r'intact\s*male',
            'min_age': 26,
            'max_age': 156
        },
        'disaster': {
            'breeds': [
                r'doberman\s*pinscher', r'german\s*shepherd', r'golden\s*retriever',
                r'bloodhound', r'rottweiler'
            ],
            'sex': r'intact\s*male',
            'min_age': 20,
            'max_age': 300
        }
    }

    cfg = rescue_map.get(rescue_type)
    if cfg is None:
        return {}

    # Breed OR search
    breed_or = []
    possible_breed_fields = ['breed', 'breed_primary', 'animal_breed']
    for field in possible_breed_fields:
        for rx in cfg['breeds']:
            breed_or.append({field: {'$regex': rx, '$options': 'i'}})

    # Sex OR search
    sex_or = []
    for field in ['sex', 'sex_upon_outcome', 'sex_upon_intake']:
        sex_or.append({field: {'$regex': cfg['sex'], '$options': 'i'}})

    # Age
    age_clause = {
        'age_upon_outcome_in_weeks': {
            "$gte": cfg['min_age'], "$lte": cfg['max_age']
        }
    }

    return {
        '$and': [
            {'$or': breed_or},
            {'$or': sex_or},
            age_clause
        ]
    }


# Updates table based on filter
@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'selected_rows'),
    Input('filter-type', 'value')
)
def update_dashboard(filter_type):
    query = build_rescue_query(filter_type)

    try:
        records = db.read(query)
    except:
        records = []

    if not isinstance(records, list):
        try:
            records = list(records)
        except:
            records = []

    if len(records) > 0:
        temp_df = pd.DataFrame.from_records(records)

        if '_id' in temp_df.columns:
            temp_df.drop(columns=['_id'], inplace=True)

        # Ensures species also exists in filtered data
        if 'animal_type' in temp_df.columns:
            temp_df['species'] = temp_df['animal_type'].apply(
                lambda x: 'Cat' if isinstance(x, str) and 'cat' in x.lower()
                else ('Dog' if isinstance(x, str) and 'dog' in x.lower() else None)
            )

        # Resets selection whenever table data changes
        return temp_df.to_dict('records'), []

    return [], []


# Breed pie chart
@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data")
)
def update_graphs(viewData):

    if viewData is None or len(viewData) == 0:
        local_df = df.copy()
    else:
        local_df = pd.DataFrame.from_dict(viewData)

    breed_col = None
    for c in ['breed', 'Breed', 'animal_breed', 'primary_breed', 'breed_primary']:
        if c in local_df.columns:
            breed_col = c
            break

    if breed_col is None:
        return [html.Div("Breed column not found.")]

    counts = (
        local_df[breed_col].fillna("Unknown")
        .value_counts()
        .reset_index()
    )
    counts.columns = [breed_col, 'count']

    fig = px.pie(
        counts,
        values='count',
        names=breed_col,
        title='Breed Distribution'
    )

    return [dcc.Graph(figure=fig, style={'height': '500px'})]


# Highlights column
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in (selected_columns or [])]


# Map callback
@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('datatable-id', "derived_virtual_selected_rows")
    ]
)
def update_map(viewData, index):

    if viewData is None:
        return html.Div("No data to display on map.")

    dff = pd.DataFrame.from_dict(viewData)

    # Displays empty map if nothing selected
    if not index:
        return [
            dl.Map(style={'width': '100%', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10,
                   children=[dl.TileLayer(), html.Div("Select a row.")])
        ]

    row = index[0]

    # Normalises latitude/longitude field names
    lat = None
    lon = None

    for c in ['location_lat', 'lat', 'latitude']:
        if c in dff.columns:
            lat = c
            break

    for c in ['location_long', 'lon', 'longitude', 'lng']:
        if c in dff.columns:
            lon = c
            break

    if lat is None or lon is None:
        return html.Div("Latitude/Longitude not found.")

    try:
        lat_val = float(dff.iloc[row][lat])
        lon_val = float(dff.iloc[row][lon])
    except:
        return html.Div("Invalid coordinates.")

    # Name + breed tooltip
    name = None
    for c in ['name', 'Name']:
        if c in dff.columns:
            name = c
            break

    breed = None
    for c in ['breed', 'Breed', 'animal_breed']:
        if c in dff.columns:
            breed = c
            break

    tooltip_text = dff.iloc[row][breed] if breed else "Animal"
    popup_name = dff.iloc[row][name] if name else "Unknown"

    return [
        dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[lat_val, lon_val],
            zoom=12,
            children=[
                dl.TileLayer(),
                dl.Marker(
                    position=[lat_val, lon_val],
                    children=[
                        dl.Tooltip(str(tooltip_text)),
                        dl.Popup([html.H4("Animal Name"), html.P(str(popup_name))])
                    ]
                )
            ]
        )
    ]


# Runs the dashboard
app.run_server(mode='external', port=8050)


Dash app running on https://miamijackson-freddiekiwi-3000.codio.io/proxy/8050/
